In [1]:
import pandas as pd
import sqlite3

In [2]:
# loading the clean data
df = pd.read_csv("../data/clean_jobs_stage1.csv")

In [3]:
# Connect to SQLite database (creates file if not exist)
conn = sqlite3.connect("../data/jobs.db")
cur = conn.cursor()

In [4]:
# Save dataframe to SQL table
df.to_sql("jobs", conn, if_exists="replace", index=False)

1200

In [5]:
# 1. Count jobs per location
query1 = "SELECT location, COUNT(*) AS job_count FROM jobs GROUP BY location ORDER BY job_count DESC;"
jobs_per_location = pd.read_sql(query1, conn)
print(jobs_per_location.head())

                 location  job_count
0                  Remote        283
1       +1 locationRemote         52
2            New York, NY         25
3  Remote in New York, NY         13
4      +2 locationsRemote         12


In [7]:
# 2. Average salary by location (remove missing salaries first)
df_salary = df[df['salary'].notnull()]
df_salary.to_sql("jobs_salary", conn, if_exists="replace", index=False)
query2 = "SELECT location, AVG(CAST(salary AS FLOAT)) AS avg_salary FROM jobs_salary GROUP BY location ORDER BY avg_salary DESC;"
avg_salary_location = pd.read_sql(query2, conn)
print(avg_salary_location.head())

                                            location  avg_salary
0                               Woonsocket, RI 02895         0.0
1  Windsor Mill, MD 21244 (Howard Park area)+3 lo...         0.0
2                                  Wichita, KS 67219         0.0
3                                       Westlake, TX         0.0
4                                 Wellston, OH 45692         0.0


In [8]:
# 3. Top companies hiring (SQL)
query3 = "SELECT company, COUNT(*) AS job_count FROM jobs GROUP BY company ORDER BY job_count DESC LIMIT 10;"
top_companies = pd.read_sql(query3, conn)
print(top_companies)

                     company  job_count
0      Artech Consulting LLC         10
1                 CVS Health          9
2     Sandhills Global, Inc.          7
3                      Ezoic          7
4                Capital One          7
5  JPMorgan Chase Bank, N.A.          6
6                      Chewy          6
7                   Cash App          6
8               Apex Systems          6
9     University of Illinois          5


In [9]:
# Close connection
conn.close()